In [2]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os


C:\Users\billy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
LLVIP_path = "YoloV5_ComputerVision/LLVIP.zip"
# !unzip -q $LLVIP_path -d YoloV5_ComputerVision/LLVIP

In [ ]:
# Download and prepare the fall detection dataset
data, info = tfds.load('fall_detection', with_info=True, as_supervised=True)
train_data, test_data = data['train'], data['test']
clear_output()


In [ ]:
# Define the YOLOv5 model architecture
model = tf.keras.models.Sequential([
    # Add the YOLOv5 backbone layers
    tf.keras.applications.YOLOv5Small(
        input_shape=(416, 416, 3), include_top=False),
    # Add a global average pooling layer
    tf.keras.layers.GlobalAveragePooling2D(),
    # Add a dense output layer for classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:

# Set up the training and validation data
train_data = train_data.map(lambda x, y: (tf.image.resize(x, (416, 416)), y))
test_data = test_data.map(lambda x, y: (tf.image.resize(x, (416, 416)), y))
train_data = train_data.batch(32).prefetch(2)
test_data = test_data.batch(32).prefetch(2)

# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print('Test accuracy:', test_acc)

# Save the trained model
model.save('fall_detection_model.h5')
